In [6]:
import utils.DatasetAccess as db_access
import utils.arguments as arg
import pandas as pd
import utils.Pruning as pruning



In [7]:
def get_data(arguments, connection, from_date, to_date):
    attribute_name = ""
    attribute_value = ""

    primary_category = db_access.get_primay_category(connection)
    secondary_category = db_access.get_secondary_category(connection)
    company_id = db_access.get_companyid(connection)

    if False:
        if ensure_valid_values(
            arguments.primarycategory, primary_category, "primary category"
        ):
            print(
                f"Models will be trained on companies with primary category in {arguments.primarycategory}"
            )
            attribute_name = "primarycategory"
            attribute_value = [f"'{x}'" for x in arguments.primarycategory]

    elif False:
        if ensure_valid_values(
            arguments.secondarycategory, secondary_category, "secondary category"
        ):
            print(
                f"Models will be trained on companies with secondary category in {arguments.secondarycategory}"
            )
            attribute_name = "secondarycategory"
            attribute_value = [f"'{x}'" for x in arguments.secondarycategory]

    elif True:
        if ensure_valid_values(
            [int(x) for x in [15515]], company_id, "companyid"
        ):
            print(
                f"Models will be trained on companies with company id in {[15515]}"
            )
            attribute_name = "identifier"
            attribute_value = [15515]

    else:
        print("No information was provided. No models will be trained.")
        return pd.DataFrame()

    data = db_access.get_data_for_attribute(
        attribute_name,
        attribute_value,
        connection,
        "T",
        from_time=from_date,
        to_time=to_date,
    )

    data = [
        d
        for d in data
        if pruning.is_there_enough_points(
            from_date, to_date, d.data.shape[0], 0.7, arguments.timeunit
        )
    ]

    # data = [d for d in data if len(d.data) > 1000]

    if arguments.limit and len(data) > arguments.limit:
        print(
            "Data is too large. Only the first {} rows will be used.".format(
                arguments.limit
            )
        )
        data = data[: arguments.limit]
    else:
        print(f"Data is {len(data)} rows long.")

    print(f"-----------------------------------------------")
    print(f"------ Introducint {len(data)} companies ------")
    print(f"-----------------------------------------------")

    for company in data:
        print(f"  - {company.name}, ({company.id}, {company.data.shape})")

    return data
def ensure_valid_values(input_values, actual_values, value_type):
    for value in input_values:
        if not value in actual_values:
            raise Exception(f"Value '{value}' is not a valid {value_type}")
    return True

In [28]:
from_date = "2021-11-20 00:00:00"
to_date = "2021-12-31 23:59:59"
connection = db_access.get_connection()

data = get_data(arguments, connection, from_date, to_date)

usage: ipykernel_launcher.py [-h] [--model {all,fb,informer,lstm,arima}]
                             [--primarycategory PRIMARYCATEGORY [PRIMARYCATEGORY ...]]
                             [--secondarycategory SECONDARYCATEGORY [SECONDARYCATEGORY ...]]
                             [--limit LIMIT] [--use_args USE_ARGS]
                             [--save_data SAVE_DATA]
                             [--companyid COMPANYID [COMPANYID ...]]
                             [--timeunit TIMEUNIT] [--time TIME]
                             [--seasonality_mode SEASONALITY_MODE]
                             [--yearly_seasonality YEARLY_SEASONALITY]
                             [--weekly_seasonality WEEKLY_SEASONALITY]
                             [--daily_seasonality DAILY_SEASONALITY]
                             [--include_history INCLUDE_HISTORY]
                             [--horizon HORIZON] [--period PERIOD]
                             [--initial INITIAL]
                             [--us

SystemExit: 2